In [1]:
import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSplitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel


In [2]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats',  'athlete_info'])
data = loader.load()

g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\data_loader.py:56: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\data_loader.py:56: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [3]:
splitter = DataSplitter(sample = 20000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [4]:
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_method': 'standard', # standard or quantile

    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero',
        'drop_missing_threshold': 0.9
        # 'neighbors': 5,
    },
    'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)


g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\feature_engineering_parts\helpers.py:306: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(
g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\feature_engineering_parts\helpers.py:265: UserWarning: Could not convert -- to time, returning as is. Error: invalid literal for int() with base 10: '--'
  warnings.warn(


In [5]:
train_preprocessed = preprocessor.transform(data=train_data)

g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\feature_engineering_parts\helpers.py:306: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(
g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\feature_engineering_parts\helpers.py:265: UserWarning: Could not convert -- to time, returning as is. Error: invalid literal for int() with base 10: '--'
  warnings.warn(


Converted height to height in imperial units
Converted weight to weight in imperial units


In [6]:
test_preprocessed = preprocessor.transform(data=test_data)

g:\my drive\ms\000.research\fall 2024\ds@gt\wod\wod-prediction\wod_predictor\feature_engineering_parts\helpers.py:306: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(


Converted height to height in imperial units
Converted weight to weight in imperial units


In [7]:
config = {}
rf_modeler = RandomForestModel(config = config, n_estimators = 10)
rf_modeler.fit(X = train_preprocessed['X'], y = train_preprocessed['y'])
rf_modeler.predict(X = test_preprocessed['X'])
rf_modeler.show_results(y_test = test_preprocessed['y'], meta_data = test_preprocessed['meta_data'])

Mean Absolute Error: 0.52
Mean Absolute Percentage Error: 2.78


,y_test_mean,y_pred_mean,error_mean,error_percentage
workout_name,,,,
workout_23.1,184.832075,185.365649,7.545030,4.082100
workout_23.1_scaled,234.400000,208.148450,33.659775,14.359972
workout_23.2A,131.850638,132.141642,11.749836,8.911475
workout_23.2B,207.477358,207.518542,18.783788,9.053416
workout_23.3,24.112896,26.246526,5.940550,24.636401
workout_23.3_scaled,9.890146,12.243936,2.535789,25.639551
